In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import os
import findspark

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'

findspark.init()

In [ ]:
spark = SparkSession.builder.appName("lets do some querying").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

In [ ]:
df = spark.read.parquet(f"/parkingviolations/raw_all.parquet").cache()

In [ ]:
df.printSchema()

## Question 1: What's the most amount that was paid in violations and by which Plate ID?

In [ ]:
filtered_df = df.filter((df.plate_id != "BLANKPLATE") & (df.plate_id != "N/A"))

violations_count = filtered_df.groupBy("plate_id", "vehicle_make").count()

top_violated_plates = violations_count.orderBy("count", ascending=False).limit(10)

top_violated_plates.show()

In [ ]:
from pyspark.sql.functions import sum, col, format_number, count

grouped_df = df.groupBy("violation_code", "violation_description") \
    .agg(sum("all_other_areas").alias("total_fine"), count("*").alias("violation_count"))

sorted_df = grouped_df.orderBy(col("total_fine").desc()) \
    .withColumn("total_fine", format_number(col("total_fine"), 0))

sorted_df.show(truncate=False)

## Question 2: At what hour of the day is it most likely to get a ticket?

## Question 3: Which percentage of the total violation "income" has been provided by the top 10% (top 10% amount of tickets)?